In [4]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision.transforms as transform
from torch.utils.data import random_split
import numpy as np
import math
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import math

#M = 1 #F=0
#Labels
#Acute stress disorder - 0
#Adjustment disorder - 1
#Alcohol use disorder - 2
#Behavioral addiction disorder - 3
#Bi-polar disorder - 4
#Depressive disorder - 5
#Healthy control - 6
#Obsessive compulsive disorder - 7
#Panic disorder - 8
#Posttraumatic stress disorder - 9
#Schizophrenia - 10
#Social anxiety disorder - 11

disorderclasses = ['Acute stress disorder', 'Adjustment disorder', 'Alcohol use disorder', 'Behavioral addiction disorder', 'Bi-polar disorder', 'Depressive disorder', 'Healthy control', 'Obsessive compulsive disorder', 'Panic disorder', 'Posttraumatic stress disorder', 'Schizophrenia', 'Social anxiety disorder']
num_epochs = 220 
batch_size = 10
learning_rate = 0.0003

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class EEGdataset(Dataset):
    def __init__(self):
        EEGdataset = pd.read_csv('./data/EEG.machinelearing_data_BRMH.csv')
        EEGdataset = EEGdataset.reset_index()
        
        EEGdata = EEGdataset.iloc[1:,:].values
        dataframe = pd.DataFrame(EEGdata)
        #dataframe.to_csv('./data/dataprog.csv', index=False)
        dataframe.iloc[:, 2] = LabelEncoder().fit_transform(dataframe.iloc[:, 2])
        dataframe.iloc[:, 8] = LabelEncoder().fit_transform(dataframe.iloc[:, 8])
        #dataframe.to_csv('./data/dataprog1.csv', index=False)
        dataframe = dataframe.drop(dataframe.columns[[0,1,3,4,5,6,7]], axis=1)
        dataframe = dataframe.astype(float)
        #dataframe.to_csv('./data/dataprog2.csv', index=False)
        dataframe = dataframe.reset_index()

        deletearrayrow = [0]
        deletearraycol = [0]
        
        for i in range(len(dataframe.index)):
            for u in range(len(dataframe.columns)):
                value = dataframe.iloc[i, u]
                if math.isnan(value) == True or value is None:
                    operatorrow = 0
                    for b in range(len(deletearrayrow)):
                        if deletearrayrow[b] == i:
                            operatorrow = 0
                            break
                        else:
                            operatorrow = 1

                    if operatorrow == 1:
                        deletearrayrow.append(i)

                    operatorcol = 0
                    for b in range(len(deletearraycol)):
                        if deletearraycol[b] == u:
                            operatorcol = 0
                            break
                        else:
                            operatorcol = 1

                    if operatorcol == 1:
                        deletearraycol.append(u)

        dataframe = dataframe.drop(dataframe.columns[[deletearraycol[1]]], axis=1)   
    
        dfsamples = dataframe.drop(dataframe.columns[[2]], axis=1)
        dflabels = dataframe.iloc[:, 2]
        
        for col in dfsamples.columns:
            dfsamples[col] = (dfsamples[col] - dfsamples[col].min())/(dfsamples[col].max() - dfsamples[col].min())

        dfsamples = dfsamples.to_numpy()
        dflabels = dflabels.to_numpy()
        
        x = torch.from_numpy(dfsamples)
        y = torch.from_numpy(dflabels)
        
        self.x = x
        self.y = y
        
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
        
    def __len__(self):
        return self.n_samples
    

data = EEGdataset()
num2 = 50
num1 = (data.n_samples) - num2

lengths = [num1, num2]
print(lengths)

train_dataset, test_dataset = random_split(data, lengths)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

#ex = iter(train_loader)
#sample, label = ex.next()

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(1142, 515)
        self.l2 = nn.Linear(515, 410)
        self.l3 = nn.Linear(410, 12)
        self.tanh = nn.Tanh()
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return x

model = NeuralNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (samples, labels) in enumerate(train_loader):
        samples = samples.to(device)
        labels = labels.to(device)
        labels = labels.long()
        samples = samples.float()
        outputs = model(samples)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print(f'Epoch: {epoch+1}, Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_classes_correct = [0 for i in range(len(disorderclasses))]
    n_classes_samples = [0 for i in range(len(disorderclasses))]

    for samples, labels in train_loader:
        samples.to(device)
        labels.to(device)
        outputs = model(samples.float())
        _, predictions = torch.max(outputs, 1)

        n_samples += labels.size(0)
        for x in range(len(labels)):
            disorder = int(torch.IntTensor.item(labels[x]))
            n_classes_samples[disorder] += 1
            if predictions[x] == labels[x]:
                n_correct += 1
                n_classes_correct[disorder] += 1
            
        
    print('______________')
    print(n_correct)
    print(n_samples)
    print(n_classes_samples)
    print(n_classes_correct)
    acc = (n_correct/ n_samples)*100
    print(acc)
    print('_____________')
    
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_classes_correct = [0 for i in range(len(disorderclasses))]
    n_classes_samples = [0 for i in range(len(disorderclasses))]

    for samples, labels in test_loader:
        samples.to(device)
        labels.to(device)
        outputs = model(samples.float())
        _, predictions = torch.max(outputs, 1)

        n_samples += labels.size(0)
        for x in range(len(labels)):
            disorder = int(torch.IntTensor.item(labels[x]))
            n_classes_samples[disorder] += 1
            if predictions[x] == labels[x]:
                n_correct += 1
                n_classes_correct[disorder] += 1
        
    print('______________')
    print(n_correct)
    print(n_samples)
    print(n_classes_samples)
    print(n_classes_correct)
    acc = (n_correct/ n_samples)*100
    print(acc)
    print('_____________')
    
        


[894, 50]
Epoch: 1, Step [10/90], Loss: 2.3589
Epoch: 1, Step [20/90], Loss: 2.4159
Epoch: 1, Step [30/90], Loss: 2.2041
Epoch: 1, Step [40/90], Loss: 2.7275
Epoch: 1, Step [50/90], Loss: 1.8932
Epoch: 1, Step [60/90], Loss: 2.2638
Epoch: 1, Step [70/90], Loss: 2.4731
Epoch: 1, Step [80/90], Loss: 2.4534
Epoch: 1, Step [90/90], Loss: 2.8300
Epoch: 2, Step [10/90], Loss: 2.3021
Epoch: 2, Step [20/90], Loss: 2.6704
Epoch: 2, Step [30/90], Loss: 2.3922
Epoch: 2, Step [40/90], Loss: 2.2339
Epoch: 2, Step [50/90], Loss: 2.3598
Epoch: 2, Step [60/90], Loss: 2.1346
Epoch: 2, Step [70/90], Loss: 2.5184
Epoch: 2, Step [80/90], Loss: 2.2174
Epoch: 2, Step [90/90], Loss: 2.3004
Epoch: 3, Step [10/90], Loss: 1.8277
Epoch: 3, Step [20/90], Loss: 2.1462
Epoch: 3, Step [30/90], Loss: 2.5249
Epoch: 3, Step [40/90], Loss: 1.9421
Epoch: 3, Step [50/90], Loss: 2.3999
Epoch: 3, Step [60/90], Loss: 2.8835
Epoch: 3, Step [70/90], Loss: 2.4778
Epoch: 3, Step [80/90], Loss: 2.4994
Epoch: 3, Step [90/90], Loss

Epoch: 25, Step [60/90], Loss: 1.4422
Epoch: 25, Step [70/90], Loss: 2.3470
Epoch: 25, Step [80/90], Loss: 1.9280
Epoch: 25, Step [90/90], Loss: 1.6492
Epoch: 26, Step [10/90], Loss: 1.8617
Epoch: 26, Step [20/90], Loss: 1.7989
Epoch: 26, Step [30/90], Loss: 1.6526
Epoch: 26, Step [40/90], Loss: 1.9395
Epoch: 26, Step [50/90], Loss: 1.8384
Epoch: 26, Step [60/90], Loss: 1.5668
Epoch: 26, Step [70/90], Loss: 2.0912
Epoch: 26, Step [80/90], Loss: 2.4728
Epoch: 26, Step [90/90], Loss: 1.8177
Epoch: 27, Step [10/90], Loss: 1.2845
Epoch: 27, Step [20/90], Loss: 1.1513
Epoch: 27, Step [30/90], Loss: 1.7972
Epoch: 27, Step [40/90], Loss: 1.9039
Epoch: 27, Step [50/90], Loss: 1.5701
Epoch: 27, Step [60/90], Loss: 1.6473
Epoch: 27, Step [70/90], Loss: 1.3838
Epoch: 27, Step [80/90], Loss: 1.5110
Epoch: 27, Step [90/90], Loss: 2.0161
Epoch: 28, Step [10/90], Loss: 2.0679
Epoch: 28, Step [20/90], Loss: 1.7219
Epoch: 28, Step [30/90], Loss: 1.6319
Epoch: 28, Step [40/90], Loss: 1.6886
Epoch: 28, S

Epoch: 49, Step [90/90], Loss: 1.0090
Epoch: 50, Step [10/90], Loss: 1.3972
Epoch: 50, Step [20/90], Loss: 1.1559
Epoch: 50, Step [30/90], Loss: 1.1143
Epoch: 50, Step [40/90], Loss: 1.4490
Epoch: 50, Step [50/90], Loss: 1.3269
Epoch: 50, Step [60/90], Loss: 1.3418
Epoch: 50, Step [70/90], Loss: 0.7524
Epoch: 50, Step [80/90], Loss: 1.2981
Epoch: 50, Step [90/90], Loss: 0.5625
Epoch: 51, Step [10/90], Loss: 1.0677
Epoch: 51, Step [20/90], Loss: 1.4192
Epoch: 51, Step [30/90], Loss: 1.6106
Epoch: 51, Step [40/90], Loss: 1.2444
Epoch: 51, Step [50/90], Loss: 0.8859
Epoch: 51, Step [60/90], Loss: 1.2552
Epoch: 51, Step [70/90], Loss: 0.8954
Epoch: 51, Step [80/90], Loss: 1.1961
Epoch: 51, Step [90/90], Loss: 0.9665
Epoch: 52, Step [10/90], Loss: 1.0836
Epoch: 52, Step [20/90], Loss: 0.7292
Epoch: 52, Step [30/90], Loss: 1.1412
Epoch: 52, Step [40/90], Loss: 1.2439
Epoch: 52, Step [50/90], Loss: 0.7763
Epoch: 52, Step [60/90], Loss: 0.8720
Epoch: 52, Step [70/90], Loss: 1.2769
Epoch: 52, S

Epoch: 74, Step [30/90], Loss: 1.0369
Epoch: 74, Step [40/90], Loss: 0.2690
Epoch: 74, Step [50/90], Loss: 0.5130
Epoch: 74, Step [60/90], Loss: 0.4957
Epoch: 74, Step [70/90], Loss: 0.4294
Epoch: 74, Step [80/90], Loss: 0.4078
Epoch: 74, Step [90/90], Loss: 1.0419
Epoch: 75, Step [10/90], Loss: 0.6167
Epoch: 75, Step [20/90], Loss: 0.4041
Epoch: 75, Step [30/90], Loss: 0.5750
Epoch: 75, Step [40/90], Loss: 0.5585
Epoch: 75, Step [50/90], Loss: 0.4977
Epoch: 75, Step [60/90], Loss: 0.5968
Epoch: 75, Step [70/90], Loss: 0.6552
Epoch: 75, Step [80/90], Loss: 0.5488
Epoch: 75, Step [90/90], Loss: 0.3468
Epoch: 76, Step [10/90], Loss: 0.5945
Epoch: 76, Step [20/90], Loss: 0.2757
Epoch: 76, Step [30/90], Loss: 0.4786
Epoch: 76, Step [40/90], Loss: 0.7454
Epoch: 76, Step [50/90], Loss: 0.9907
Epoch: 76, Step [60/90], Loss: 0.5276
Epoch: 76, Step [70/90], Loss: 0.6834
Epoch: 76, Step [80/90], Loss: 0.3460
Epoch: 76, Step [90/90], Loss: 0.1872
Epoch: 77, Step [10/90], Loss: 0.9120
Epoch: 77, S

Epoch: 98, Step [40/90], Loss: 0.2647
Epoch: 98, Step [50/90], Loss: 0.1226
Epoch: 98, Step [60/90], Loss: 0.0449
Epoch: 98, Step [70/90], Loss: 0.3323
Epoch: 98, Step [80/90], Loss: 0.4842
Epoch: 98, Step [90/90], Loss: 0.1247
Epoch: 99, Step [10/90], Loss: 0.1718
Epoch: 99, Step [20/90], Loss: 0.1619
Epoch: 99, Step [30/90], Loss: 0.1157
Epoch: 99, Step [40/90], Loss: 0.0295
Epoch: 99, Step [50/90], Loss: 0.2527
Epoch: 99, Step [60/90], Loss: 0.1269
Epoch: 99, Step [70/90], Loss: 0.1957
Epoch: 99, Step [80/90], Loss: 0.2171
Epoch: 99, Step [90/90], Loss: 0.4274
Epoch: 100, Step [10/90], Loss: 0.1809
Epoch: 100, Step [20/90], Loss: 0.0637
Epoch: 100, Step [30/90], Loss: 0.0700
Epoch: 100, Step [40/90], Loss: 0.2164
Epoch: 100, Step [50/90], Loss: 0.0676
Epoch: 100, Step [60/90], Loss: 0.1775
Epoch: 100, Step [70/90], Loss: 0.1381
Epoch: 100, Step [80/90], Loss: 0.0954
Epoch: 100, Step [90/90], Loss: 0.1689
Epoch: 101, Step [10/90], Loss: 0.0705
Epoch: 101, Step [20/90], Loss: 0.1073
E

Epoch: 121, Step [90/90], Loss: 0.0191
Epoch: 122, Step [10/90], Loss: 0.0193
Epoch: 122, Step [20/90], Loss: 0.0316
Epoch: 122, Step [30/90], Loss: 0.0352
Epoch: 122, Step [40/90], Loss: 0.0265
Epoch: 122, Step [50/90], Loss: 0.0287
Epoch: 122, Step [60/90], Loss: 0.0280
Epoch: 122, Step [70/90], Loss: 0.0194
Epoch: 122, Step [80/90], Loss: 0.0332
Epoch: 122, Step [90/90], Loss: 0.0306
Epoch: 123, Step [10/90], Loss: 0.0235
Epoch: 123, Step [20/90], Loss: 0.0450
Epoch: 123, Step [30/90], Loss: 0.0433
Epoch: 123, Step [40/90], Loss: 0.0310
Epoch: 123, Step [50/90], Loss: 0.0167
Epoch: 123, Step [60/90], Loss: 0.0156
Epoch: 123, Step [70/90], Loss: 0.0440
Epoch: 123, Step [80/90], Loss: 0.0121
Epoch: 123, Step [90/90], Loss: 0.0736
Epoch: 124, Step [10/90], Loss: 0.0413
Epoch: 124, Step [20/90], Loss: 0.0275
Epoch: 124, Step [30/90], Loss: 0.0138
Epoch: 124, Step [40/90], Loss: 0.0352
Epoch: 124, Step [50/90], Loss: 0.0407
Epoch: 124, Step [60/90], Loss: 0.0263
Epoch: 124, Step [70/90],

Epoch: 145, Step [50/90], Loss: 0.0145
Epoch: 145, Step [60/90], Loss: 0.0084
Epoch: 145, Step [70/90], Loss: 0.0109
Epoch: 145, Step [80/90], Loss: 0.0128
Epoch: 145, Step [90/90], Loss: 0.0092
Epoch: 146, Step [10/90], Loss: 0.0077
Epoch: 146, Step [20/90], Loss: 0.0180
Epoch: 146, Step [30/90], Loss: 0.0149
Epoch: 146, Step [40/90], Loss: 0.0101
Epoch: 146, Step [50/90], Loss: 0.0137
Epoch: 146, Step [60/90], Loss: 0.0118
Epoch: 146, Step [70/90], Loss: 0.0194
Epoch: 146, Step [80/90], Loss: 0.0253
Epoch: 146, Step [90/90], Loss: 0.0155
Epoch: 147, Step [10/90], Loss: 0.0049
Epoch: 147, Step [20/90], Loss: 0.0113
Epoch: 147, Step [30/90], Loss: 0.0106
Epoch: 147, Step [40/90], Loss: 0.0095
Epoch: 147, Step [50/90], Loss: 0.0063
Epoch: 147, Step [60/90], Loss: 0.0132
Epoch: 147, Step [70/90], Loss: 0.0157
Epoch: 147, Step [80/90], Loss: 0.0072
Epoch: 147, Step [90/90], Loss: 0.0050
Epoch: 148, Step [10/90], Loss: 0.0128
Epoch: 148, Step [20/90], Loss: 0.0089
Epoch: 148, Step [30/90],

Epoch: 169, Step [20/90], Loss: 0.0064
Epoch: 169, Step [30/90], Loss: 0.0034
Epoch: 169, Step [40/90], Loss: 0.0031
Epoch: 169, Step [50/90], Loss: 0.0026
Epoch: 169, Step [60/90], Loss: 0.0019
Epoch: 169, Step [70/90], Loss: 0.0029
Epoch: 169, Step [80/90], Loss: 0.0053
Epoch: 169, Step [90/90], Loss: 0.0007
Epoch: 170, Step [10/90], Loss: 0.0032
Epoch: 170, Step [20/90], Loss: 0.0046
Epoch: 170, Step [30/90], Loss: 0.0053
Epoch: 170, Step [40/90], Loss: 0.0040
Epoch: 170, Step [50/90], Loss: 0.0065
Epoch: 170, Step [60/90], Loss: 0.0030
Epoch: 170, Step [70/90], Loss: 0.0039
Epoch: 170, Step [80/90], Loss: 0.0059
Epoch: 170, Step [90/90], Loss: 0.0019
Epoch: 171, Step [10/90], Loss: 0.0029
Epoch: 171, Step [20/90], Loss: 0.0073
Epoch: 171, Step [30/90], Loss: 0.0025
Epoch: 171, Step [40/90], Loss: 0.0030
Epoch: 171, Step [50/90], Loss: 0.0029
Epoch: 171, Step [60/90], Loss: 0.0104
Epoch: 171, Step [70/90], Loss: 0.0027
Epoch: 171, Step [80/90], Loss: 0.0021
Epoch: 171, Step [90/90],

Epoch: 192, Step [60/90], Loss: 0.0044
Epoch: 192, Step [70/90], Loss: 0.0071
Epoch: 192, Step [80/90], Loss: 0.0074
Epoch: 192, Step [90/90], Loss: 0.0168
Epoch: 193, Step [10/90], Loss: 0.0136
Epoch: 193, Step [20/90], Loss: 0.0053
Epoch: 193, Step [30/90], Loss: 0.0048
Epoch: 193, Step [40/90], Loss: 0.0063
Epoch: 193, Step [50/90], Loss: 0.0071
Epoch: 193, Step [60/90], Loss: 0.0123
Epoch: 193, Step [70/90], Loss: 0.0022
Epoch: 193, Step [80/90], Loss: 0.0076
Epoch: 193, Step [90/90], Loss: 0.0057
Epoch: 194, Step [10/90], Loss: 0.0093
Epoch: 194, Step [20/90], Loss: 0.0117
Epoch: 194, Step [30/90], Loss: 0.0110
Epoch: 194, Step [40/90], Loss: 0.0034
Epoch: 194, Step [50/90], Loss: 0.0062
Epoch: 194, Step [60/90], Loss: 0.0119
Epoch: 194, Step [70/90], Loss: 0.0020
Epoch: 194, Step [80/90], Loss: 0.0052
Epoch: 194, Step [90/90], Loss: 0.0058
Epoch: 195, Step [10/90], Loss: 0.0079
Epoch: 195, Step [20/90], Loss: 0.0087
Epoch: 195, Step [30/90], Loss: 0.0100
Epoch: 195, Step [40/90],

Epoch: 216, Step [40/90], Loss: 0.0040
Epoch: 216, Step [50/90], Loss: 0.0028
Epoch: 216, Step [60/90], Loss: 0.0056
Epoch: 216, Step [70/90], Loss: 0.0023
Epoch: 216, Step [80/90], Loss: 0.0028
Epoch: 216, Step [90/90], Loss: 0.0005
Epoch: 217, Step [10/90], Loss: 0.0053
Epoch: 217, Step [20/90], Loss: 0.0032
Epoch: 217, Step [30/90], Loss: 0.0016
Epoch: 217, Step [40/90], Loss: 0.0040
Epoch: 217, Step [50/90], Loss: 0.0061
Epoch: 217, Step [60/90], Loss: 0.0032
Epoch: 217, Step [70/90], Loss: 0.0025
Epoch: 217, Step [80/90], Loss: 0.0014
Epoch: 217, Step [90/90], Loss: 0.0030
Epoch: 218, Step [10/90], Loss: 0.0020
Epoch: 218, Step [20/90], Loss: 0.0015
Epoch: 218, Step [30/90], Loss: 0.0023
Epoch: 218, Step [40/90], Loss: 0.0021
Epoch: 218, Step [50/90], Loss: 0.0034
Epoch: 218, Step [60/90], Loss: 0.0042
Epoch: 218, Step [70/90], Loss: 0.0013
Epoch: 218, Step [80/90], Loss: 0.0021
Epoch: 218, Step [90/90], Loss: 0.0031
Epoch: 219, Step [10/90], Loss: 0.0022
Epoch: 219, Step [20/90],